<a href="https://colab.research.google.com/github/kbyph/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Reading package lists... Done


In [16]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-10 04:43:13--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  1.58MB/s    in 0.6s    

2022-08-10 04:43:14 (1.58 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [17]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [18]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33670092|R1UUISQ1GKOJTI|B00EE62UAE|     583436067|Trademark Home Po...|            Home|          1|            0|          0|   N|                Y|                 Run|Don't buy it clap...| 2015-08-31|
|         US|   13726692|R1HOJ9WE8VCVOD|B001APXO5C|     465035091|O2-Cool 10-Inch P...|            Home|          5|    

### Create DataFrames to match tables

In [19]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
vine_reviews_clean_df = df.dropna()
vine_reviews_clean_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33670092|R1UUISQ1GKOJTI|B00EE62UAE|     583436067|Trademark Home Po...|            Home|          1|            0|          0|   N|                Y|                 Run|Don't buy it clap...| 2015-08-31|
|         US|   13726692|R1HOJ9WE8VCVOD|B001APXO5C|     465035091|O2-Cool 10-Inch P...|            Home|          5|    

In [20]:
# Create the customers_table DataFrame
# customers_df = df.groupby("").agg({""}).withColumnRenamed("", "customer_count")
customers_df = vine_reviews_clean_df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   28241302|             1|
|   15991768|             1|
|   17067926|             7|
|    8905239|             3|
|   51794064|             1|
|   33132797|             3|
|   21325555|             1|
|    1574612|             1|
|   12819130|             3|
|   44224073|             1|
|   16679709|             1|
|   10742726|             1|
|   52391013|             2|
|   38209321|            10|
|    2545142|             1|
|   52918362|             6|
|   47663761|             3|
|   25031530|            21|
|   29476112|             1|
|   51496479|             1|
+-----------+--------------+
only showing top 20 rows



In [21]:
# Create the products_table DataFrame and drop duplicates. 
# products_df = df.select([]).drop_duplicates()
products_df = vine_reviews_clean_df.select(["product_id","product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0000635WI|Leachco Snoogle O...|
|B00B14LI5U|Stretchy Beading ...|
|B00JOTC23Q|Coats Yarn Red He...|
|B00FXOFLXO|Lasko 1843 18" Re...|
|B00V3PU6QE|Estilo Mini Count...|
|B00BLY6POE|HOUSWEETY 12pc Mi...|
|B00JRH758M|SHARP Refurbished...|
|B00DUVTYDU|InterDesign Tor T...|
|B00HOWO7E2|Wieco Art - White...|
|B00M669V9W|Good Grips Compac...|
|B00HIZK5EQ|Splash Home 4-Pie...|
|B0118OQJHO|Pink Box Metallic...|
|B00GU8QKNW|One Vietnamese So...|
|B002V9195S|Pro-Mart DAZZ 42-...|
|B009I1CLKO|Crystal Allies Ma...|
|B00QD8LF20|The Big One® Supe...|
|B001DHXU9C|Dimensions Needle...|
|B00LGZVMSM|Wood Carving Chis...|
|B00MHW49ZG|Blue Donuts Multi...|
|B0042TMUC6|LA Linen 54-Inch ...|
+----------+--------------------+
only showing top 20 rows



In [22]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
# review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df = vine_reviews_clean_df.select(["review_id","customer_id","product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df = review_id_df.orderBy(review_id_df["review_date"].desc())
review_id_df.show(truncate=False)

+--------------+-----------+----------+--------------+-----------+
|review_id     |customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1UUISQ1GKOJTI|33670092   |B00EE62UAE|583436067     |2015-08-31 |
|R1HOJ9WE8VCVOD|13726692   |B001APXO5C|465035091     |2015-08-31 |
|RDNGVXMWQN2TN |50131396   |B002HFDLCK|136507891     |2015-08-31 |
|R3OM9COQMVTDJ2|16046884   |B00PL9EFPQ|631238459     |2015-08-31 |
|R3OFUQVR4Y80Q9|11417282   |B00B5TPVQS|190431573     |2015-08-31 |
|R2GD25SBBGRKPO|48013430   |B00NQDGHDC|124050883     |2015-08-31 |
|R3RRSLYKVWH9WB|7341516    |B00I3BYEXM|933053612     |2015-08-31 |
|R2HMFAGJJU6NT3|20696954   |B008QZD7RY|77515396      |2015-08-31 |
|R2QDRZATHBY4GY|28241302   |B00TQ6MXE0|267158600     |2015-08-31 |
|R107946YZK57Q2|45444347   |B004O39RJ4|473994651     |2015-08-31 |
|R31EK6FUI5YAL1|15080335   |B003LZ09C0|945333576     |2015-08-31 |
|R2PCO0R2FKDQLD|12081067   |B00I56KQV4|383995956     |2015-08-

In [23]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = vine_reviews_clean_df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1UUISQ1GKOJTI|          1|            0|          0|   N|                Y|
|R1HOJ9WE8VCVOD|          5|            9|          9|   N|                Y|
| RDNGVXMWQN2TN|          5|            0|          0|   N|                Y|
|R3OM9COQMVTDJ2|          2|            0|          0|   N|                Y|
|R3OFUQVR4Y80Q9|          4|            0|          0|   N|                N|
|R2GD25SBBGRKPO|          5|            0|          0|   N|                Y|
|R3RRSLYKVWH9WB|          5|            0|          0|   N|                Y|
|R2HMFAGJJU6NT3|          5|            0|          0|   N|                Y|
|R2QDRZATHBY4GY|          1|            0|          0|   N|                Y|
|R107946YZK57Q2|          2|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [24]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://m16challenge.cmlnhnipagnp.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "M16challenge", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)